In [4]:
import numpy as np
import vtk
import vmtk
from vmtk.vmtkscripts import vmtkCenterlines


def point_cloud_to_watertight_mesh(point_cloud_file, output_mesh_file, alpha=0.1):
    """Converts a point cloud (.npy) to a watertight mesh using alpha shapes."""
    # Load the point cloud
    points = np.load(point_cloud_file)

    # Create a vtkPoints object and add the points
    vtk_points = vtk.vtkPoints()
    for point in points:
        vtk_points.InsertNextPoint(point)

    # Create a vtkPolyData object to hold the points
    poly_data = vtk.vtkPolyData()
    poly_data.SetPoints(vtk_points)

    # Perform Delaunay triangulation with alpha shapes
    delaunay = vtk.vtkDelaunay3D()
    delaunay.SetInputData(poly_data)
    delaunay.SetAlpha(alpha)
    delaunay.Update()

    # Extract the surface mesh
    surface_filter = vtk.vtkDataSetSurfaceFilter()
    surface_filter.SetInputConnection(delaunay.GetOutputPort())
    surface_filter.Update()

    # Write the watertight mesh to a file
    writer = vtk.vtkXMLPolyDataWriter()
    writer.SetFileName(output_mesh_file)
    writer.SetInputConnection(surface_filter.GetOutputPort())
    writer.Write()

    print(f"Watertight mesh saved to: {output_mesh_file}")

def mesh_to_volume(input_mesh_file, volume_file, spacing=(1.0, 1.0, 1.0)):
    """Converts a watertight mesh to a volumetric representation (e.g., voxel grid)."""
    # Read the watertight mesh
    reader = vtk.vtkXMLPolyDataReader()
    reader.SetFileName(input_mesh_file)
    reader.Update()

    # Convert the mesh to a volume
    bounds = reader.GetOutput().GetBounds()
    image_data = vtk.vtkImageData()
    image_data.SetSpacing(spacing)

    # Compute the dimensions of the volume
    dims = [
        int((bounds[1] - bounds[0]) / spacing[0]),
        int((bounds[3] - bounds[2]) / spacing[1]),
        int((bounds[5] - bounds[4]) / spacing[2]),
    ]
    image_data.SetDimensions(dims)
    image_data.SetOrigin(bounds[0], bounds[2], bounds[4])

    # Use a stencil to fill the volume
    stencil = vtk.vtkPolyDataToImageStencil()
    stencil.SetInputConnection(reader.GetOutputPort())
    stencil.SetOutputSpacing(spacing)
    stencil.SetOutputOrigin(bounds[0], bounds[2], bounds[4])
    stencil.SetOutputWholeExtent(image_data.GetExtent())
    stencil.Update()

    # Apply the stencil to create the volume
    volume = vtk.vtkImageStencil()
    volume.SetInputData(image_data)
    volume.SetStencilConnection(stencil.GetOutputPort())
    volume.ReverseStencilOff()
    volume.SetBackgroundValue(0)  # Outside the mesh
    volume.Update()

    # Write the volume to a file
    writer = vtk.vtkXMLImageDataWriter()
    writer.SetFileName(volume_file)
    writer.SetInputConnection(volume.GetOutputPort())
    writer.Write()

    print(f"Volume saved to: {volume_file}")

def visualize_mesh_with_centerline(mesh_file, centerline_file):
    """Visualizes a mesh and its centerline together."""
    # Read the mesh from the file
    mesh_reader = vtk.vtkXMLPolyDataReader()
    mesh_reader.SetFileName(mesh_file)
    mesh_reader.Update()

    # Read the centerline from the file
    centerline_reader = vtk.vtkXMLPolyDataReader()
    centerline_reader.SetFileName(centerline_file)
    centerline_reader.Update()

    # Check if the mesh and centerline are valid
    if mesh_reader.GetOutput().GetNumberOfPoints() == 0:
        print("Error: The mesh file is empty or invalid.")
        return
    if centerline_reader.GetOutput().GetNumberOfPoints() == 0:
        print("Error: The centerline file is empty or invalid.")
        return

    # Create a mapper and actor for the mesh
    mesh_mapper = vtk.vtkPolyDataMapper()
    mesh_mapper.SetInputConnection(mesh_reader.GetOutputPort())

    mesh_actor = vtk.vtkActor()
    mesh_actor.SetMapper(mesh_mapper)
    mesh_actor.GetProperty().SetOpacity(0.3)  # Make the mesh semi-transparent

    # Create a mapper and actor for the centerline
    centerline_mapper = vtk.vtkPolyDataMapper()
    centerline_mapper.SetInputConnection(centerline_reader.GetOutputPort())

    centerline_actor = vtk.vtkActor()
    centerline_actor.SetMapper(centerline_mapper)
    centerline_actor.GetProperty().SetColor(1.0, 0.0, 0.0)  # Set the centerline color to red
    centerline_actor.GetProperty().SetLineWidth(3)  # Make the centerline thicker

    # Create a renderer, render window, and interactor
    renderer = vtk.vtkRenderer()
    renderer.AddActor(mesh_actor)
    renderer.AddActor(centerline_actor)
    renderer.SetBackground(0.1, 0.1, 0.1)  # Dark background

    render_window = vtk.vtkRenderWindow()
    render_window.AddRenderer(renderer)

    render_window_interactor = vtk.vtkRenderWindowInteractor()
    render_window_interactor.SetRenderWindow(render_window)

    # Render and start interaction
    render_window.Render()
    render_window_interactor.Start()

def visualize_points_on_mesh(mesh, points):
    """Visualizes points on a mesh."""
    point_polydata = vtk.vtkPolyData()
    vtk_points = vtk.vtkPoints()
    for point in points:
        vtk_points.InsertNextPoint(point)
    point_polydata.SetPoints(vtk_points)

    point_mapper = vtk.vtkPolyDataMapper()
    point_mapper.SetInputData(point_polydata)

    point_actor = vtk.vtkActor()
    point_actor.SetMapper(point_mapper)
    point_actor.GetProperty().SetColor(1.0, 0.0, 0.0)  # Red points
    point_actor.GetProperty().SetPointSize(10)

    # Visualize the mesh and points together
    mesh_mapper = vtk.vtkPolyDataMapper()
    mesh_mapper.SetInputData(mesh)

    mesh_actor = vtk.vtkActor()
    mesh_actor.SetMapper(mesh_mapper)
    mesh_actor.GetProperty().SetOpacity(0.3)  # Semi-transparent mesh

    renderer = vtk.vtkRenderer()
    renderer.AddActor(mesh_actor)
    renderer.AddActor(point_actor)

    render_window = vtk.vtkRenderWindow()
    render_window.AddRenderer(renderer)

    render_window_interactor = vtk.vtkRenderWindowInteractor()
    render_window_interactor.SetRenderWindow(render_window)

    render_window.Render()
    render_window_interactor.Start()

def clean_and_connect_mesh(input_mesh):
    """Cleans the mesh and extracts the largest connected component."""
    connectivity_filter = vtk.vtkConnectivityFilter()
    connectivity_filter.SetInputData(input_mesh)
    connectivity_filter.SetExtractionModeToLargestRegion()
    connectivity_filter.Update()
    return connectivity_filter.GetOutput()

def smooth_mesh(input_mesh, iterations=30, relaxation_factor=0.1):
    """Smooths the mesh to improve quality."""
    smoother = vtk.vtkSmoothPolyDataFilter()
    smoother.SetInputData(input_mesh)
    smoother.SetNumberOfIterations(iterations)
    smoother.SetRelaxationFactor(relaxation_factor)
    smoother.FeatureEdgeSmoothingOff()
    smoother.BoundarySmoothingOn()
    smoother.Update()
    return smoother.GetOutput()

def extract_centerline(input_mesh_file, centerline_file):
    """
    Extracts the centerline of a tubular structure using vmtk's vmtkCenterlines.
    """
    # Read the watertight mesh
    reader = vtk.vtkXMLPolyDataReader()
    reader.SetFileName(input_mesh_file)
    reader.Update()
    polydata = reader.GetOutput()

    # Check if the input mesh is valid
    if polydata.GetNumberOfPoints() == 0:
        print("Error: Input mesh is empty or invalid.")
        return

    # Clean and smooth the mesh
    polydata = clean_and_connect_mesh(polydata)
    polydata = smooth_mesh(polydata)

    # Compute the center of mass of the mesh
    center_of_mass_filter = vtk.vtkCenterOfMass()
    center_of_mass_filter.SetInputData(polydata)
    center_of_mass_filter.Update()
    center_of_mass = center_of_mass_filter.GetCenter()

    # Use the center of mass as the source point
    source_point = center_of_mass

    # Find the farthest point along the Z-axis for the target
    bounds = polydata.GetBounds()
    target_point = [(bounds[0] + bounds[1]) / 2, (bounds[2] + bounds[3]) / 2, bounds[5]]  # Top center

    # Find the closest points on the mesh to the source and target
    locator = vtk.vtkPointLocator()
    locator.SetDataSet(polydata)
    locator.BuildLocator()

    source_id = locator.FindClosestPoint(source_point)
    target_id = locator.FindClosestPoint(target_point)

    # Get the coordinates of the closest points
    source_coords = polydata.GetPoint(source_id)
    target_coords = polydata.GetPoint(target_id)

    # Visualize the source and target points
    visualize_points_on_mesh(polydata, [source_coords, target_coords])

    # Flatten the source and target points into a single list
    source_points = list(source_coords)
    target_points = list(target_coords)

    # Extract the centerline using vmtkCenterlines
    centerline_filter = vmtkCenterlines()
    centerline_filter.Surface = polydata
    centerline_filter.SeedSelectorName = 'pointlist'
    centerline_filter.SourcePoints = source_points
    centerline_filter.TargetPoints = target_points
    centerline_filter.Execute()

    # Extract the centerline as a polydata
    centerline_polydata = centerline_filter.Centerlines

    # Check if the centerline has points
    if centerline_polydata.GetNumberOfPoints() == 0:
        print("Error: Centerline generation failed.")
        return

    # Debug: Print the number of points and lines in the centerline
    print(f"Number of points in centerline: {centerline_polydata.GetNumberOfPoints()}")
    print(f"Number of lines in centerline: {centerline_polydata.GetNumberOfCells()}")

    # Write the centerline to a file
    writer = vtk.vtkXMLPolyDataWriter()
    writer.SetFileName(centerline_file)
    writer.SetInputData(centerline_polydata)
    writer.Write()

    print(f"Centerline saved to: {centerline_file}")

if __name__ == "__main__":
    # Input and output file paths
    point_cloud_file = r"C:\Users\Lenovo\OneDrive - Syddansk Universitet\Dokumenter\GitHub\Broncho-Project\test\accumulated_point_cloud_frame_1100.npy"
    output_mesh_file = r"C:\Users\Lenovo\OneDrive - Syddansk Universitet\Dokumenter\GitHub\Broncho-Project\test\output_mesh.vtp"
    centerline_file = r"C:\Users\Lenovo\OneDrive - Syddansk Universitet\Dokumenter\GitHub\Broncho-Project\test\centerline.vtp"

    # Step 1: Convert point cloud to watertight mesh
    point_cloud_to_watertight_mesh(point_cloud_file, output_mesh_file, alpha=0.1)

    # Step 2: Extract the centerline
    extract_centerline(output_mesh_file, centerline_file)

    # Step 3: Visualize the mesh and centerline together
    visualize_mesh_with_centerline(output_mesh_file, centerline_file)

Watertight mesh saved to: C:\Users\Lenovo\OneDrive - Syddansk Universitet\Dokumenter\GitHub\Broncho-Project\test\output_mesh.vtp
Cleaning surface.
Triangulating surface.
Computing centerlines.
Computing centerlines...Number of points in centerline: 2
Number of lines in centerline: 1
Centerline saved to: C:\Users\Lenovo\OneDrive - Syddansk Universitet\Dokumenter\GitHub\Broncho-Project\test\centerline.vtp


In [20]:
import vtk
import numpy as np

def slice_mesh_and_connect_centroids(mesh_file, num_slices=10):
    """
    Slices a mesh along the x-axis, calculates centroids for each slice,
    and connects the centroids with a line. Visualizes the line within the mesh.
    """
    # Read the mesh from the file
    reader = vtk.vtkXMLPolyDataReader()
    reader.SetFileName(mesh_file)
    reader.Update()
    mesh = reader.GetOutput()

    # Get the bounds of the mesh along the x-axis
    bounds = mesh.GetBounds()
    x_min, x_max = bounds[0], bounds[1]
    slice_width = (x_max - x_min) / num_slices

    centroids = []

    # Slice the mesh and calculate centroids
    for i in range(num_slices):
        x_start = x_min + i * slice_width
        x_end = x_start + slice_width

        # Extract points within the current slice
        points_in_slice = []
        for j in range(mesh.GetNumberOfPoints()):
            point = mesh.GetPoint(j)
            if x_start <= point[0] < x_end:
                points_in_slice.append(point)

        if points_in_slice:
            # Calculate the centroid of the slice
            points_in_slice = np.array(points_in_slice)
            centroid = points_in_slice.mean(axis=0)
            centroids.append(centroid)

    # Convert centroids to vtkPoints
    vtk_centroids = vtk.vtkPoints()
    for centroid in centroids:
        vtk_centroids.InsertNextPoint(centroid)

    # Create a vtkPolyLine to connect the centroids
    polyline = vtk.vtkPolyLine()
    polyline.GetPointIds().SetNumberOfIds(len(centroids))
    for i in range(len(centroids)):
        polyline.GetPointIds().SetId(i, i)

    # Create a vtkCellArray to hold the polyline
    cells = vtk.vtkCellArray()
    cells.InsertNextCell(polyline)

    # Create a vtkPolyData to hold the centroids and the polyline
    line_polydata = vtk.vtkPolyData()
    line_polydata.SetPoints(vtk_centroids)
    line_polydata.SetLines(cells)

    # Create a mapper and actor for the line
    line_mapper = vtk.vtkPolyDataMapper()
    line_mapper.SetInputData(line_polydata)

    line_actor = vtk.vtkActor()
    line_actor.SetMapper(line_mapper)
    line_actor.GetProperty().SetColor(1.0, 0.0, 0.0)  # Red line
    line_actor.GetProperty().SetLineWidth(2)

    # Create a mapper and actor for the mesh
    mesh_mapper = vtk.vtkPolyDataMapper()
    mesh_mapper.SetInputData(mesh)

    mesh_actor = vtk.vtkActor()
    mesh_actor.SetMapper(mesh_mapper)
    mesh_actor.GetProperty().SetColor(0.8, 0.8, 0.8)  # Light gray mesh
    mesh_actor.GetProperty().SetOpacity(0.3)  # Semi-transparent mesh

    # Create a renderer, render window, and interactor
    renderer = vtk.vtkRenderer()
    renderer.AddActor(mesh_actor)  # Add the mesh actor
    renderer.AddActor(line_actor)  # Add the line actor
    renderer.SetBackground(0.1, 0.1, 0.1)  # Dark background

    render_window = vtk.vtkRenderWindow()
    render_window.AddRenderer(renderer)

    render_window_interactor = vtk.vtkRenderWindowInteractor()
    render_window_interactor.SetRenderWindow(render_window)

    # Render and start interaction
    render_window.Render()
    render_window_interactor.Start()


# File path to the mesh
output_mesh_file = r"C:\Users\Lenovo\OneDrive - Syddansk Universitet\Dokumenter\GitHub\Broncho-Project\test\output_mesh.vtp"

# Call the function
slice_mesh_and_connect_centroids(output_mesh_file, num_slices=10)